In [43]:
import numpy as np
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import datetime as datetime

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel,RBF,Matern

import kernels

In [44]:
%load_ext autoreload
%autoreload 2
import solution
from solution import Model
from solution import cv_eval

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#def main():
train_x_name = "train_x.csv"
train_y_name = "train_y.csv"

train_x = np.loadtxt(train_x_name, delimiter=',')
train_y = np.loadtxt(train_y_name, delimiter=',')

# load the test dateset
test_x_name = "test_x.csv"
test_x = np.loadtxt(test_x_name, delimiter=',')

In [4]:
df_vals = np.stack([train_x[:,0],train_x[:,1],train_y],axis=1)

df = pd.DataFrame(data = df_vals,columns = ['x0','x1','y'])

df_left = df[df['x0']<-0.5]
df_left = df_left.sample(frac=0.1,random_state=42)

train_x = df_left[['x0','x1']].values
train_y = df_left['y'].values

print(df_left.shape)

(1710, 3)


In [6]:
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, Matern, RBF

#kernels_grid = [kernels.sklearn_best()]
kernels_grid = [
    RBF(),
    1.0*RBF()+WhiteKernel(),
    1.0*Matern(nu=1.5)+WhiteKernel(),
    1.0*Matern(nu=2.5)+WhiteKernel(),
    1.0+1.0*Matern(nu=1.5)+WhiteKernel(),
    1.0+1.0*Matern(nu=2.5)+WhiteKernel()
]
K_cv = 3
model_config = {
    "use_skit_learn":True,
    "use_nystrom":False,
    "use_fitc" : False,
    "model_preprocess_left_frac":1.0
}

result_csv_path = "cv_results.csv"
tune_HP_nested_CV2(kernels_grid, K_cv, model_config,result_csv_path)

outer=0
kernel=RBF(length_scale=1)
(1368, 3)
i=0
logging_setup
training cost fn   0.051183
val cost fn        7.114097
val log likelihood fn        -319859718.017057


i=1
logging_setup
training cost fn   0.051183
val cost fn        7.114097
val log likelihood fn        -319859718.017057


i=2
logging_setup
training cost fn   0.051183
val cost fn        7.114097
val log likelihood fn        -319859718.017057


[-3.19859718e+08 -3.19859718e+08 -3.19859718e+08]
logging_setup
4.7014748839108
7.1140974392303065
0.0
outer=0
kernel=1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1)
(1368, 3)
i=0
logging_setup
training cost fn   0.054964
val cost fn        0.054739
val log likelihood fn        1282.128277


i=1
logging_setup
training cost fn   0.054964
val cost fn        0.054739
val log likelihood fn        1282.128277


i=2
logging_setup
training cost fn   0.054964
val cost fn        0.054739
val log likelihood fn        1282.128277


[1282.12827689 1282.12827689 1282.12827689]
logging

training cost fn   0.054886
val cost fn        0.054828
val log likelihood fn        1304.997638


i=1
logging_setup
training cost fn   0.054886
val cost fn        0.054828
val log likelihood fn        1304.997638


i=2
logging_setup
training cost fn   0.054886
val cost fn        0.054828
val log likelihood fn        1304.997638


[1304.99763847 1304.99763847 1304.99763847]
logging_setup
0.05422163083821756
0.0548281980038977
0.0
outer=2
kernel=1**2 + 1**2 * Matern(length_scale=1, nu=1.5) + WhiteKernel(noise_level=1)
(1368, 3)
i=0
logging_setup
training cost fn   0.054739
val cost fn        0.054676
val log likelihood fn        1316.767041


i=1
logging_setup
training cost fn   0.054739
val cost fn        0.054676
val log likelihood fn        1316.767041


i=2
logging_setup
training cost fn   0.054739
val cost fn        0.054676
val log likelihood fn        1316.767041


[1316.767041 1316.767041 1316.767041]
logging_setup
0.0542163091794487
0.05467580928583735
0.0
outer=2
kernel=1**2 +

{'inner': array([[7.11409744, 0.05473902, 0.05434152, 0.05434833, 0.05430638,
         0.05428859],
        [6.54786258, 0.0556628 , 0.055803  , 0.05569099, 0.05583626,
         0.05577697],
        [5.80282766, 0.05521408, 0.05471379, 0.0548282 , 0.05467581,
         0.0547553 ],
        [5.61962401, 0.05482316, 0.05454212, 0.05459698, 0.05449809,
         0.05457101],
        [7.37203828, 0.05520349, 0.05496575, 0.05502393, 0.05495984,
         0.05499809]]), 'outer': []}

In [23]:
params = fitted.kernel_.get_params()

In [65]:
def tune_HP_nested_CV(kernels_grid, K_cv, model_config):
    train_x_split, test_x_split, train_y_split, test_y_split = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
 
    CV_res = []
    for hp_kernel in kernels_grid:
        model_config["kernel"] = hp_kernel
        cv_m = cv_eval(cv_splits = K_cv,
               cv_preprocess_left_frac = 0.05,
               model_config=model_config)
        val_cost_array = cv_m.run_cross_validation(train_x_split, train_y_split)
        print(np.mean(val_cost_array))
        print(np.std(val_cost_array))
        CV_res.append(np.mean(val_cost_array))
    return CV_res

print(np.shape(train_x))
#nested n1*k*n2 total
def tune_HP_nested_LML(kernel_tunable, K_cv, model_config,result_csv_path):
    n_outer = 5
    n_inner = K_cv
    cv_config = {"outer":n_outer,"kernels":None,"inner":n_inner}
    
    # automatic Hyperparameters optimisation with scikit-learn 
    model_config["use_skit_learn"] = True
    model_config["use_nystrom"] = False
    model_config["use_fitc"] = False
    model_config["kernel"] = kernel_tunable
    grp = Model(model_config)
    
    cv_outer = KFold(n_splits=n_outer, shuffle=True, random_state=42)
    outer_res = []
    lml_array = np.array([])
    
    outer_count = 0
    for train_ix,test_ix in cv_outer.split(train_x):
        train_x_split,test_x_split = train_x[train_ix,:],train_x[test_ix,:]
        train_y_split,test_y_split = train_y[train_ix],train_y[test_ix]
        #train_x_split, test_x_split, train_y_split, test_y_split = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
 
     
        cv_result_cols = ["test_cost","val_cost","val_lml","init_kernel",
                          "left_data","cv_config","model_config","time"]
        df_cv_result = pd.DataFrame(columns = cv_result_cols)
        #df_cv_result = pd.read_csv(result_csv_path)
        print("outer=%d"%outer_count)

        # Fit model on test set 
        grp.fit_model(train_x = train_x_split, train_y = train_y_split)
        val_llikelihood = grp.likelihood()
        print("val log likelihood fn        %f"%(val_llikelihood))
        lml_array = np.append(lml_array,val_llikelihood)
        #print(fitted.kernel_.get_params())

        # Evaluate model on test set 
        test_y_pred = fitted.predict(test_x_split)
        test_cost = solution.cost_function(test_y_split,test_y_pred)
        outer_res.append(test_cost)
        print("cost on test set{}".format(test_cost))

        df_cv_result = df_cv_result.append({
                      "test_cost":test_cost,
                      "val_cost": None,
                      "val_lml":np.mean(lml_array),
                      "init_kernel":str(kernel_tunable),
                      "left_data":int(df_left.shape[0]),
                      "cv_config":str(cv_config),
                      "model_config":str(model_config),
                      "time":datetime.datetime.now()
                     },ignore_index=True)
        df_cv_result.to_csv(result_csv_path,index=False,header=False,mode="a")
       
        outer_count += 1
    print("Mean cost on test set{}".format(np.mean(outer_res)))
    return {"outer":outer_res}

(1710, 2)


In [66]:
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, Matern, RBF

#kernels_grid = [kernels.sklearn_best()]
kernels_tunable = 1.0*RBF()+WhiteKernel()+ 1.0*Matern()

K_cv = 3
model_config = {
    "use_skit_learn":True,
    "use_nystrom":False,
    "use_fitc" : False,
    "model_preprocess_left_frac":1.0
}

result_csv_path = "/Users/laurieprelot/Documents/Projects/2020_probabilistic_AI/task1_handout/data/LML_results.csv"
tune_HP_nested_LML(kernels_tunable, K_cv, model_config,result_csv_path)

outer=0
logging_setup
val log likelihood fn        2003.808234
cost on test set0.02509003071653513
outer=1
logging_setup
val log likelihood fn        2008.041991
cost on test set0.03299163092893364
outer=2
logging_setup
val log likelihood fn        2016.687575
cost on test set0.05217101283888147
outer=3
logging_setup
val log likelihood fn        1986.545652
cost on test set0.01563895435744038
outer=4
logging_setup
val log likelihood fn        2003.253255
cost on test set0.0223846310600029
Mean cost on test set0.029655251980358704


{'outer': [0.02509003071653513,
  0.03299163092893364,
  0.05217101283888147,
  0.01563895435744038,
  0.0223846310600029]}

In [46]:
# Unchanged 

In [45]:
def logging_setup():
    print("logging_setup")
    project_dir = os.path.dirname('/Users/laurieprelot/Documents/Projects/2020_probabilistic_AI/task1_handout/data/log_file')
    logging_path = os.path.join(*[project_dir,'train.log'])
    logging.basicConfig(filename=logging_path, filemode='a',\
            format='%(asctime)s - %(name)s - %(message)s', level=logging.INFO)
    logging.info("Begin logging for single training run")
    root = logging.getLogger()
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    root.addHandler(handler)